In [14]:
import cv2
import numpy as np

# 读取图片
image1 = cv2.imread('3.jpg')
image2 = cv2.imread('4.jpg')

# 转换为灰度图像
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# 使用SIFT检测关键点和描述符
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

# 使用FLANN匹配器进行特征匹配
flann_index_kdtree = 1
index_params = dict(algorithm=flann_index_kdtree, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(descriptors1, descriptors2, k=2)

# 保留良好的匹配
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

# 获取匹配点对的坐标
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# 计算单应性矩阵
H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# 应用单应性变换
result = cv2.warpPerspective(image1, H, (image2.shape[1], image2.shape[0]))
print(H)
# 显示结果
cv2.imwrite('result_H.png', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 8.79366395e-01 -2.99741795e-01  2.82243275e+02]
 [ 3.52698760e-01  1.09124829e+00 -1.82939343e+02]
 [-5.27048048e-05  1.35075106e-04  1.00000000e+00]]
